<H3>Trabalho Prático 1 - Algoritmos 2 2024.1 UFMG</H3>
Membros:

* Gabriel Arcanjo Campelo Fadoul - 2021031645

In [ ]:
import plotly.express as px

<H4> Carga das instâncias de testes

<H4>Funções Primitivas de Geometria Computacional

<H4>Funções de Triangulação

<H4>Animações